# Regression

In [1]:
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
data = pd.read_csv("Dataset/house-price/housing.csv")


In [3]:
data

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


In [4]:
data = pd.get_dummies(data, columns=['ocean_proximity'], drop_first=True,dtype=int)

In [5]:
y = data['median_house_value']
X = data.drop(["median_house_value"],axis=1)

In [6]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [7]:
X_train,X_val,y_train,y_val = train_test_split(X_train,y_train,test_size=0.2)

In [8]:
sc =  StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_val = sc.transform(X_val)

In [9]:
model_a =  keras.models.Sequential([
    keras.layers.Dense(50,activation="relu"),
    keras.layers.Dense(10,activation="relu"),
    keras.layers.Dense(1)
])

In [10]:
model_a.compile(loss="mean_squared_error",
              optimizer="sgd",
              metrics=["mean_absolute_error"]
              )

In [11]:
# model_a.fit(X_train,y_train,epochs=40,validation_data=(X_val,y_val))

In [12]:
# model_a.evaluate(X_test,y_test)

## Functional API
<hr>

In [13]:
input_  = keras.layers.Input(shape= X_train.shape[1:])
hidden_layer1 = keras.layers.Dense(50,activation="relu")(input_)
hidden_layer2 = keras.layers.Dense(10,activation="relu")(hidden_layer1)
concatenate_layer = keras.layers.Concatenate()([input_,hidden_layer2])
output = keras.layers.Dense(1)(concatenate_layer)
model_b = keras.Model(inputs=[input_],outputs=[output])

In [14]:
model_b.compile(loss="mean_squared_error",
              optimizer="sgd",
              metrics=["mean_absolute_error"]
              )

In [15]:
# model_b.fit(X_train,y_train,epochs=40,validation_data=(X_val,y_val))


In [16]:
# model_b.evaluate(X_test,y_test)

## Multi-input
<hr>

In [17]:
X_train_in_1 , X_train_in_2 = X_train[:,:6],X_train[:,-4:]
X_val_in_1 , X_val_in_2 = X_val[:,:6],X_val[:,-4:]
X_test_in_1 , X_test_in_2 = X_test[:,:6],X_test[:,-4:]




In [18]:
input_1  = keras.layers.Input(shape=[6])
input_2  = keras.layers.Input(shape=[4])
hidden_layer1 = keras.layers.Dense(50,activation="relu")(input_1)
hidden_layer2 = keras.layers.Dense(10,activation="relu")(hidden_layer1)
concatenate_layer = keras.layers.Concatenate()([input_2,hidden_layer2])
output = keras.layers.Dense(1)(concatenate_layer)
model_c = keras.Model(inputs=[input_1,input_2],outputs=[output])

In [19]:
model_c.compile(loss="mean_squared_error",
              optimizer="sgd",
              metrics=["mean_absolute_error"]
              )

In [20]:
# model_c.fit( (X_train_in_1,X_train_in_2) , y_train ,epochs=40 , validation_data=( (X_val_in_1,X_val_in_2) ,y_val))


## Multi-output
<hr>

In [21]:
input_1  = keras.layers.Input(shape=[6])
input_2  = keras.layers.Input(shape=[4])
hidden_layer1 = keras.layers.Dense(50,activation="relu")(input_1)
hidden_layer2 = keras.layers.Dense(10,activation="relu")(hidden_layer1)
concatenate_layer = keras.layers.Concatenate()([input_2,hidden_layer2])
output = keras.layers.Dense(1,name="output")(concatenate_layer)
helper_output = keras.layers.Dense(1,name="helper-output")(hidden_layer2)



model_d = keras.Model(inputs=[input_1,input_2],outputs=[output,helper_output])

In [22]:
model_d.compile(loss=["mse","mae"],
              loss_weights = [0.8,0.2],
              optimizer="sgd",
              metrics=[['mean_absolute_error'], ['mean_absolute_error']] 
              )

In [23]:
model_d.fit( (X_train_in_1,X_train_in_2) , (y_train,y_train) ,epochs=40 , validation_data=( (X_val_in_1,X_val_in_2) ,(y_val,y_val)))


Epoch 1/40
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - helper-output_loss: 214188.2969 - helper-output_mean_absolute_error: 214212.9844 - loss: 78499458864659205510263734272.0000 - output_loss: 98072340951172198550728605696.0000 - output_mean_absolute_error: 30737403215872.0000 - val_helper-output_loss: 210302.1719 - val_helper-output_mean_absolute_error: 210886.7500 - val_loss: 92848443703285321302016.0000 - val_output_loss: 116113525967923783401472.0000 - val_output_mean_absolute_error: 322793734144.0000
Epoch 2/40
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - helper-output_loss: 206494.0625 - helper-output_mean_absolute_error: 206489.1094 - loss: 268865330099582838767616.0000 - output_loss: 335903824482392942313472.0000 - output_mean_absolute_error: 92017123328.0000 - val_helper-output_loss: 210301.3438 - val_helper-output_mean_absolute_error: 210885.9062 - val_loss: 167869424310421553152.0000 - val_output_loss: 209757949802361913344.0000 - val_output_mean_absolute_error: 14402018304.00

##  Sub-Class API
<hr>

In [24]:
class WideAndDeepANN (keras.Model):
    def __init__(self,activation="relu",**kwargs):
        super().__init__(**kwargs)
        self.hidden_layer1 = keras.layers.Dense(50,activation=activation)
        self.hidden_layer2 = keras.layers.Dense(10,activation=activation)
        self.final_output = keras.layers.Dense(1)
        self.helper_output = keras.layers.Dense(1)

    def call(self,inputs):
        input_1,input_2 = inputs
        h1_out = self.hidden_layer1(input_1)
        h2_out = self.hidden_layer2(h1_out)
        concat_out = keras.layers.concatenate([input_2,h2_out])
        final_output  = self.final_output(concat_out)
        helper_output  = self.helper_output(h2_out)
        return final_output,helper_output
    
    



In [25]:
model_f = WideAndDeepANN()

In [26]:
model_f.compile(loss=["mse","mae"],
              loss_weights = [0.8,0.2],
              optimizer="sgd",
              metrics=[['mean_absolute_error'], ['mean_absolute_error']] 
              )

In [27]:
# model_f.fit( (X_train_in_1,X_train_in_2) , (y_train,y_train) ,epochs=40 , validation_data=( (X_val_in_1,X_val_in_2) ,(y_val,y_val)))


## Saving the model
<hr>

In [ ]:
model_d.save("models/model_d.h5")
# .save_weights() ===> sub-class API

In [30]:
model_d_load = keras.models.load_model("models/model_d.h5", compile=False)